<a href="https://colab.research.google.com/github/RobinSmits/Dutch-LLMs/blob/main/Open_Llama_13B_Alpaca_Clean_Dutch_Qlora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
import os
from google.colab import drive
drive.mount('/content/drive')

# Set Folder to use...
WORK_DIR = '/content/drive/My Drive/LLM/OpenLlama13BAlpacaCleanDutchQlora/'
os.makedirs(WORK_DIR, exist_ok = True)

Mounted at /content/drive


In [2]:
# Install required packages
!pip install accelerate
!pip install sentencepiece
!pip install bitsandbytes
!pip install transformers
!pip install -U git+https://github.com/huggingface/peft.git
!pip install datasets
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.8 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-7l9p77k9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-7l9p77k9
  Resolved https://github.com/huggingface/peft.git to commit 032fff92fb74b737a2934e91a08d82142fb79dc3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
# Import Modules
from datasets import load_dataset, load_from_disk
from huggingface_hub import notebook_login
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, TaskType
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig
import torch
import transformers
import wandb


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [ ]:
# WandB logging
reporting = "wandb"

# WandB Login
wandb.login()

# WandB Settings
wandb.init(project = 'OpenLlama13BAlpacaCleanDutch',
           config = {"model": 'open_llama_13B', "dataset": 'alpaca-dutch-cleaned'})

In [5]:
# Set Name Constants
model_name = "openlm-research/open_llama_13b"
hf_model_name = 'open_llama_13b_alpaca_clean_dutch_qlora'

In [6]:
# Create Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          use_fast = False,
                                          add_eos_token = True)
tokenizer.pad_token_id = 0

# Max Length
MAX_LEN = 512

# Tokenizer Summary
print(tokenizer)

# Special Tokens
print('pad_token_id: ',tokenizer.pad_token_id)
print('bos_token_id: ',tokenizer.bos_token_id)
print('eos_token_id: ',tokenizer.eos_token_id)
print('pad_token_id: ',tokenizer.decode(tokenizer.pad_token_id))
print('bos_token_id: ',tokenizer.decode(tokenizer.bos_token_id))
print('eos_token_id: ',tokenizer.decode(tokenizer.eos_token_id))

LlamaTokenizer(name_or_path='openlm-research/open_llama_13b', vocab_size=32000, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False)
pad_token_id:  0
bos_token_id:  1
eos_token_id:  2
pad_token_id:  <unk>
bos_token_id:  <s>
eos_token_id:  </s>


In [7]:
# Create Config
config = AutoConfig.from_pretrained(model_name,
                                    use_cache = False)

# Summary
config

LlamaConfig {
  "_name_or_path": "openlm-research/open_llama_13b",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 5120,
  "initializer_range": 0.02,
  "intermediate_size": 13824,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 40,
  "num_hidden_layers": 40,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.30.2",
  "use_cache": false,
  "vocab_size": 32000
}

In [8]:
# Create Model
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             config = config,
                                             device_map = {"":0},
                                             quantization_config = BitsAndBytesConfig(load_in_4bit = True,
                                                                                  bnb_4bit_use_double_quant = True,
                                                                                  bnb_4bit_quant_type = 'nf4',
                                                                                  bnb_4bit_compute_dtype = torch.bfloat16))

# Enable Gradient Checkpointing
model.gradient_checkpointing_enable()

# Show Model Summary
print(model)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [9]:
# Create LoRA config
loraconfig = LoraConfig(r = 8,
                        lora_alpha = 16,
                        target_modules = ['q_proj', 'v_proj'],
                        lora_dropout = 0.05,
                        bias = 'none',
                        task_type = TaskType.CAUSAL_LM)

# Prep for Training
model = prepare_model_for_kbit_training(model)

# Create LoRA Model
model = get_peft_model(model, loraconfig)
model.print_trainable_parameters()

# Show Model Summary
print(model)

trainable params: 6,553,600 || all params: 6,678,533,120 || trainable%: 0.09812933292752765
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 5120, padding_idx=0)
        (layers): ModuleList(
          (0-39): 40 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=5120, out_features=5120, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
        

In [10]:
"""
# Alpaca Translated
datasets = load_dataset('BramVanroy/alpaca-cleaned-dutch')

# Summary
print(datasets)

# Validation size
VAL_SIZE = 2048

# Split in Train and Val datasets
dataset_split = datasets["train"].train_test_split(test_size = VAL_SIZE,
                                                   shuffle = True,
                                                   seed = 42)

# Train and Val datasets
train_data = dataset_split["train"]
val_data = dataset_split["test"]

# Save for later use
val_data.save_to_disk(f'{WORK_DIR}val_data')
train_data.save_to_disk(f'{WORK_DIR}train_data')
"""

# Load Train and Validation sets from earlier training run...use code above to generate new train and val datasets.
val_data = load_from_disk(f'{WORK_DIR}val_data')
train_data = load_from_disk(f'{WORK_DIR}train_data')

In [11]:
# Model Training Prompt
def generate_prompt(data_point):
    if data_point["input"]:
        return f"""### Instructie:
{data_point["instruction"]}

### Invoer:
{data_point["input"]}

### Antwoord:
{data_point["output"]}"""
    else:
        return f"""### Instructie:
{data_point["instruction"]}

### Antwoord:
{data_point["output"]}"""

In [12]:
# Plot some samples
counter = 0
for item_data in train_data:
    print('\n=======================================================================')
    print(generate_prompt(item_data))

    counter += 1
    if counter > 4:
        break


### Instructie:
Maak een zin met 5 uitdrukkingen.

### Antwoord:
Hij was zo boos dat hij stond te trappelen, op het verkeerde spoor zat, een appeltje te schillen had, te vroeg was begonnen, en zijn woorden moest inslikken.

### Instructie:
Voeg 3 interessante feiten over Colombia toe.

### Antwoord:
1. Colombia heeft na Brazilië de op één na grootste biodiversiteit ter wereld, met meer dan 79.000 diersoorten en 40.000 plantensoorten.
2. Colombia produceert meer koffie dan welk ander land ter wereld dan ook.
3. Colombia is het enige Zuid-Amerikaanse land met een kustlijn aan zowel de Stille Oceaan als de Caribische Zee.

### Instructie:
Geef een voorbeeld van een gebruikstoepassing voor Machine Learning

### Antwoord:
Een voorbeeld van een gebruikstoepassing voor Machine Learning is een spamfilteringsysteem. Het maakt gebruik van ML-algoritmen om spam-e-mails te detecteren en uit de inbox van een gebruiker te filteren.

### Instructie:
Herschrijf de zin om de wezelwoorden te vermijden 

In [13]:
train_data = train_data.shuffle().map(lambda x: tokenizer.encode_plus(generate_prompt(x), truncation = True, max_length = MAX_LEN, padding = "max_length"))

Map:   0%|          | 0/49664 [00:00<?, ? examples/s]

In [14]:
val_data = val_data.shuffle().map(lambda x: tokenizer.encode_plus(generate_prompt(x), truncation = True, max_length = MAX_LEN, padding = "max_length"))

Map:   0%|          | 0/2048 [00:00<?, ? examples/s]

In [15]:
# Set Steps
eval_steps = 128
save_steps = 128
logging_steps = 16

# Config Trainer
trainer = transformers.Trainer(model = model,
                               train_dataset = train_data,
                               eval_dataset = val_data,
                               args = transformers.TrainingArguments(num_train_epochs = 2,
                                                                     learning_rate = 2.0e-4,
                                                                     evaluation_strategy = "steps",
                                                                     logging_steps = logging_steps,
                                                                     save_strategy = "steps",
                                                                     eval_steps = eval_steps,
                                                                     save_steps = save_steps,
                                                                     save_total_limit = 3,
                                                                     per_device_train_batch_size = 32,
                                                                     per_device_eval_batch_size = 32,
                                                                     gradient_accumulation_steps = 2,
                                                                     warmup_steps = 64,
                                                                     fp16 = True,
                                                                     output_dir = "outputs",
                                                                     optim = "paged_adamw_8bit",
                                                                     report_to = reporting),
                               data_collator = transformers.DataCollatorForLanguageModeling(tokenizer,
                                                                                            mlm = False))

# Perform Training
trainer.train()

Step,Training Loss,Validation Loss
128,1.141800,1.142239
256,1.111100,1.106115
384,1.095600,1.099293
512,1.057900,1.079856
640,1.062900,1.060519
768,1.043900,1.049627
896,0.982900,1.037903
1024,1.005900,1.029499
1152,0.978800,1.015532
1280,0.958100,1.004862


TrainOutput(global_step=1552, training_loss=1.0727190885347189, metrics={'train_runtime': 38459.1874, 'train_samples_per_second': 2.583, 'train_steps_per_second': 0.04, 'total_flos': 1.987864898222162e+18, 'train_loss': 1.0727190885347189, 'epoch': 2.0})

In [16]:
# Save Finetuned Model
trainer.model.save_pretrained(f'{WORK_DIR}')
tokenizer.save_pretrained(f'{WORK_DIR}')

('/content/drive/My Drive/LLM/OpenLlama13BAlpacaCleanDutchQlora/tokenizer_config.json',
 '/content/drive/My Drive/LLM/OpenLlama13BAlpacaCleanDutchQlora/special_tokens_map.json',
 '/content/drive/My Drive/LLM/OpenLlama13BAlpacaCleanDutchQlora/tokenizer.model',
 '/content/drive/My Drive/LLM/OpenLlama13BAlpacaCleanDutchQlora/added_tokens.json')

In [17]:
# Huggingface Hub Login
notebook_login()

In [18]:
# Push Model to HF Hub
model.push_to_hub(hf_model_name, private = True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/26.3M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/robinsmits/open_llama_13b_alpaca_clean_dutch_qlora/commit/dd65caf095a6f603efca44bddffa25ce80e03d64', commit_message='Upload model', commit_description='', oid='dd65caf095a6f603efca44bddffa25ce80e03d64', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
# Push Tokenizer to HF Hub
tokenizer.push_to_hub(hf_model_name, private = True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/534k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/robinsmits/open_llama_13b_alpaca_clean_dutch_qlora/commit/23b96c1f8f8317f4ee93f08d098704e84ae7f7de', commit_message='Upload tokenizer', commit_description='', oid='23b96c1f8f8317f4ee93f08d098704e84ae7f7de', pr_url=None, pr_revision=None, pr_num=None)